In [1]:
!pip install transformers
!pip install datasets
!pip install mlflow
!pip install torch

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 550.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 k

In [2]:

!pip install pyngrok -q

In [3]:

import subprocess
from pyngrok import ngrok, conf
import getpass

In [6]:
import os
import mlflow
import mlflow.pytorch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
from transformers import EarlyStoppingCallback
import torch

In [7]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI])

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', 'sqli...>

In [20]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment("duration-prediction-experiment")

<Experiment: artifact_location='/content/mlruns/1', creation_time=1728775149077, experiment_id='1', last_update_time=1728775149077, lifecycle_stage='active', name='duration-prediction-experiment', tags={}>

In [21]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()
port=5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel "https://4c38-34-75-140-112.ngrok-free.app" -> "http://127.0.0.1:5000"


In [22]:
import os
import mlflow
import mlflow.pytorch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
from transformers import EarlyStoppingCallback
import torch

os.makedirs("mlruns", exist_ok=True)
mlflow.end_run()

mlflow.start_run()

data_files = 'cleaned_creative_writing_dataset.csv'
dataset = load_dataset('csv', data_files=data_files)
dataset = dataset['train'].remove_columns(['text'])
dataset = dataset.rename_column('cleaned_text', 'text')

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    input_ids = tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=32,
    )
    input_ids['labels'] = input_ids['input_ids'].copy()
    return input_ids

tokenized_datasets = dataset.map(tokenize_function, batched=True)
train_test_split = tokenized_datasets.train_test_split(test_size=0.2)
tokenized_datasets = DatasetDict({
    'train': train_test_split['train'],
    'validation': train_test_split['test']
})


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    accuracy = (predictions == labels).mean()
    return {'accuracy': accuracy}


learning_rate = 2e-5
per_device_train_batch_size = 1
num_train_epochs = 1
max_length = 32

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    no_cuda=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

mlflow.log_param("data_files", data_files)
mlflow.log_param("learning_rate", learning_rate)
mlflow.log_param("per_device_train_batch_size", per_device_train_batch_size)
mlflow.log_param("num_train_epochs", num_train_epochs)
mlflow.log_param("max_length", max_length)
mlflow.log_param("model_name", "gpt2")

trainer.train()

model.save_pretrained('./fine_tuned_gpt2')
tokenizer.save_pretrained('./fine_tuned_gpt2')

mlflow.pytorch.log_model(model, "fine_tuned_gpt2")

eval_metrics = trainer.evaluate()

if 'loss' in trainer.state.log_history[-1]:
    train_loss = trainer.state.log_history[-1]['loss']
else:
    train_loss = None

mlflow.log_metric("Training Loss", train_loss if train_loss is not None else 0.0)
mlflow.log_metric("Validation Loss", eval_metrics['eval_loss'])
mlflow.log_metric("Accuracy", eval_metrics.get('eval_accuracy', 0.0))

mlflow.end_run()

print("Model training and saving completed.")

from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "./fine_tuned_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.eval()

def generate_story(prompt, max_length=1000, temperature=1.5, top_k=100):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            do_sample=True,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_story = tokenizer.decode(output[0], skip_special_tokens=True)

    mlflow.start_run()
    mlflow.log_param("generation_max_length", max_length)
    mlflow.log_param("temperature", temperature)
    mlflow.log_param("top_k", top_k)
    mlflow.log_param("prompt", prompt)
    mlflow.end_run()

    return generated_story

prompt = "Write a story about a girl's adventures in a magical forest where she finds strange creatures"
generated_text = generate_story(prompt, max_length=1000)
print(generated_text)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1540: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
2024/10/13 02:50:15 ERROR mlflow.utils.async_logging.async_logging_queue: Run Id b7c26c61f0c549afbd3b631714897a58: Failed to log run data: Exception: Changing param valu

Epoch,Training Loss,Validation Loss,Accuracy
1,5.602600,5.522341,0.164226


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].
2024/10/13 03:40:24 WARNING mlflow.utils.requirements_utils: Found torch version (2.4.1+cu121) contains a local version label (+cu121). MLflow logged a pip requirement for this package as 'torch==2.4.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2024/10/13 03:40:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


Model training and saving completed.
Write a story about a girl's adventures in a magical forest where she finds strange creatures always found dreams may find courage use her book "It goes well story" saying first time looking pachyderm ever wrote wrote story felt i had never done novel several people asked storyboard storyboard artist kelvia kartan first introduced rozja work first book made dylan shulke book gave kartan good first time author josh duet received first issue new set introduced first issue early one morning roaz wilman introduced character kartan idea dream like girl rozi is born alittle staig minning best writers ryan james raul branly kartan called book became first ever econ one reading co write volume chronicles series characters love laura pfry best novel storyboard cartoon rozi used cartoon writing kartan wanted author to do companion series


In [24]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.8/319.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.6/436.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/13

In [26]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the fine-tuned GPT-2 model and tokenizer
model_name = "./fine_tuned_gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.eval()

# Define the generation function
def generate_story(prompt, max_length=1000, temperature=1.5, top_k=100):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    with torch.no_grad():
        output = model.generate(
            input_ids,
            max_length=max_length,
            temperature=temperature,
            top_k=top_k,
            do_sample=True,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_story = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_story

# Define Gradio interface
def gradio_generate(prompt):
    generated_text = generate_story(prompt)
    return generated_text

# Create Gradio interface
gradio_interface = gr.Interface(
    fn=gradio_generate,
    inputs="text",
    outputs="text",
    title="Story Generator",
    description="Enter a prompt to generate a story using the fine-tuned GPT-2 model.",
)

# Launch the interface
gradio_interface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fc0d79ba43be8fbfba.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
